In [3]:
import os
import h5py
import numpy as np
import matplotlib.pyplot as plt
from skimage.metrics import structural_similarity
from typing import Optional
import tensorflow as tf

reconstruction_path = '/usr/local/micapollo01/MIC/DATA/STUDENTS/mvhave7/Results/Reconstructions/BaselineUNet/Epoch6/reconstructions/'
GT_path = '/usr/local/micapollo01/MIC/DATA/SHARED/NYU_FastMRI/multicoil_test_full/'
file = 'file_brain_AXFLAIR_200_6002441.h5'

GT_path = os.path.join(GT_path,file)
reconstruction_path = os.path.join(reconstruction_path,file)

GT = h5py.File(GT_path, 'r')
reconstruction = h5py.File(reconstruction_path, 'r')


In [4]:
GT = GT['reconstruction_rss']
reconstruction = reconstruction['reconstruction']


In [5]:
GT = np.array(GT)
reconstruction = np.array(reconstruction)


In [20]:
def ssim(
    gt: np.ndarray, pred: np.ndarray, maxval: Optional[float] = None
) -> np.ndarray:
    """Compute Structural Similarity Index Metric (SSIM)"""
    if not gt.ndim == 3:
        raise ValueError("Unexpected number of dimensions in ground truth.")
    if not gt.ndim == pred.ndim:
        raise ValueError("Ground truth dimensions does not match pred.")

    maxval = gt.max() if maxval is None else maxval

    ssim = np.array([0])
    for slice_num in range(gt.shape[0]):
        #ssim = ssim + structural_similarity(
        #    gt[slice_num], pred[slice_num], data_range=maxval
        #)
        ssim = ssim + structural_similarity(
            gt[slice_num], pred[slice_num], data_range=maxval,
            gaussian_weights=True,
            sigma=0.1,
            win_size=3
        )

    return ssim / gt.shape[0]


In [18]:
def ssim_tf(gt, pred, maxval=None):
    maxval = gt.max() if maxval is None else maxval
    gt_tf = tf.convert_to_tensor(gt)
    pred_tf = tf.convert_to_tensor(pred)
    ssim = tf.reduce_mean(tf.image.ssim(img1=gt_tf,img2=pred_tf,max_val=maxval,filter_size=3,filter_sigma=0.1))
    #ssim = tf.reduce_mean(tf.image.ssim(img1=gt_tf,img2=pred_tf,max_val=maxval))
    return ssim


In [21]:
ssim(GT, reconstruction)


array([0.75772222])

In [22]:
ssim_tf(GT, reconstruction)


<tf.Tensor: shape=(), dtype=float32, numpy=0.76378495>